In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
glove_dir='../input/embeddings/glove.840B.300d/glove.840B.300d.txt'

In [ ]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [ ]:
pip install --upgrade keras

In [ ]:
train=pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv')
test_df=pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')
sub=pd.read_csv('/kaggle/input/quora-insincere-questions-classification/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
print(train.shape)
print(test_df.shape)

In [ ]:
!unzip /kaggle/input/quora-insincere-questions-classification/embeddings.zip

In [ ]:
!ls ../input/embeddings/

In [ ]:
train_df,val_df=train_test_split(train,test_size=0.1)
embedd_size=300
max_features=50000
max_len=100

In [ ]:
train_x=train_df['question_text'].fillna('__na__').values
val_x=val_df['question_text'].fillna('__na__').values
test_x=test_df['question_text'].fillna('__na__').values

In [ ]:
print(train_x.shape)

In [ ]:
tokenizer=Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_df))
train_x=tokenizer.texts_to_sequences(train_x)
val_x=tokenizer.texts_to_sequences(val_x)
test_x=tokenizer.texts_to_sequences(test_x)

In [ ]:
train_x=pad_sequences(train_x,maxlen=max_len)
val_x=pad_sequences(val_x,maxlen=max_len)
test_x=pad_sequences(test_x,maxlen=max_len)

In [ ]:
train_out=train_df['target'].values
val_out=val_df['target'].values

In [ ]:
inp=Input(shape=(max_len))
x=Embedding(max_features,embedd_size)(inp)
x=Bidirectional(LSTM(128,return_sequences=True))(x)
x=GlobalMaxPool1D()(x)
x=Dense(16,activation='relu')(x)
x=Dropout(0.2)(x)
x=Dense(1,activation='sigmoid')(x)
model=Model(inputs=inp,outputs=x)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [ ]:
print(train_x.shape)

In [ ]:
model.fit(train_x,train_out,batch_size=256,epochs=2,validation_data=(val_x,val_out))

In [ ]:
pred_y = model.predict([val_x], batch_size=256, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_out, (pred_y>thresh).astype(int))))

In [ ]:
pred_y=model.predict([test_x],batch_size=256)

In [ ]:
out_df = pd.DataFrame({"qid":test_df["qid"].values})
out_df['prediction'] = pred_y
out_df.to_csv("submission.csv", index=False)